# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Big Data Real-Time Analytics com Python e Spark</font>

## <font color='blue'>Mini-Projeto 7</font>

### <font color='blue'>Sistema de Recomendação em Tempo Real com Machine Learning, PySpark, Spark Streaming e Kafka</font>

![title](imagens/MP7.png)

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.13


In [2]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
#!pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [3]:
# https://kafka-python.readthedocs.io/en/master/
# Instala o conector do Kafka
!pip install -q kafka-python

In [4]:
# Imports
import time
import random
import kafka
import numpy as np
import pandas as pd
from json import dumps
from kafka import KafkaProducer
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

Author: Data Science Academy

kafka : 2.0.2
numpy : 1.22.4
pandas: 1.3.4



In [6]:
# Endereço do servidor Kafka
SERVER = 'localhost:9092'

In [7]:
# Nome do tópico
TOPIC = "dsaminiprojeto7"

In [8]:
# Carregamos o conjunto de dados de músicas
df_dsaminiprojeto7 = pd.read_csv("dados/dataset.csv")

In [9]:
# Ajustamos o formato de 3 colunas importantes: order_id, Artist Name(s) e Artist IDs
df_dsaminiprojeto7['order_id'] = np.arange(len(df_dsaminiprojeto7))
df_dsaminiprojeto7['Artist Name(s)'] = df_dsaminiprojeto7['Artist Name(s)'].str.replace('[^a-zA-Z]', '')
df_dsaminiprojeto7['Artist IDs'] = df_dsaminiprojeto7['Artist IDs'].str.replace('[^a-zA-Z]', '')

In [10]:
df_dsaminiprojeto7.shape

(4399, 24)

In [11]:
df_dsaminiprojeto7.head(10)

,Spotify ID,Artist IDs,Track Name,Album Name,Artist Name(s),Release Date,Duration (ms),Popularity,Added By,Added At,...,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Time Signature,order_id
0,22a0Ji6EQKkY0tBohlN4Od,qLyYYhSlsjwymwVKwW,There You Are,There You Are,KirstenLudwig,2018-08-06,231240,2,spotify:user:predict0,2018-08-28T19:51:58Z,...,-5.596,0,0.0304,0.334000,0.282000,0.1050,0.316,129.856,4,0
1,4J39ZEbwqHwtWLImUKmrn9,CRfAxYjJsDBHwvWFnjaRRRPXwFwQmoTNqNHBGU,88 Days,Heat,SaraKingIanOlney,2018-08-04,227961,8,spotify:user:predict0,2018-08-28T19:51:58Z,...,-10.749,1,0.0333,0.134000,0.582000,0.1340,0.233,155.062,4,1
2,0a12d4HUjOmQSqHqLopWYx,hytHTGTflktWAhKcxQ,Castaway,Castaway,ARZLEE,2018-08-10,230000,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.290,1,0.0314,0.110000,0.000032,0.1190,0.290,83.988,4,2
3,4u1DykFW1HjYAGNoDCiXfC,WjyoJHRHlTbUTZTwqpAgeqmtJlARXjon,Arouse,Arouse,Shagabondgoodboynoah,2018-08-03,213913,30,spotify:user:predict0,2018-08-28T19:51:58Z,...,-6.066,1,0.4330,0.072800,0.000000,0.3680,0.533,91.961,4,3
4,0u7JZm9ORerlZnnxxSdMwl,AdKmjgFzpcTvmVfGwR,Lonely,Lonely,Hayleau,2018-08-10,258738,21,spotify:user:predict0,2018-08-28T19:51:58Z,...,-3.921,0,0.0406,0.016900,0.000630,0.0542,0.577,98.954,4,4
5,0wuy2BYIVLbflFDqnR9Jay,kCwrYUFSJCubbbnZrE,Orsay,Strange Affairs,TheSvens,2018-08-03,413658,6,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.858,1,0.0316,0.048600,0.886000,0.1280,0.283,122.992,4,5
6,6LkIZZRrPQIbHMyBR5mTc2,TOsWuafqeWtrvYXqbnYAV,Nurture,Comrade,IslandFox,2018-08-09,191641,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.415,0,0.0504,0.015900,0.639000,0.1810,0.266,133.925,4,6
7,5U27fxNSd27XtX876xUsfV,HsKUExgNcRJojPmBcNqzgwpvzedAIjuDBM,Dinosaur Hair - Remix,Dinosaur Hair,AndyFerroCharlieConway,2018-08-10,257152,1,spotify:user:predict0,2018-08-28T19:51:58Z,...,-10.086,1,0.0383,0.456000,0.920000,0.1350,0.663,125.908,4,7
8,5ogJOpmyDsvrAdttU6JLnN,gslbnQQLLcNzfjnxQY,Breathing Underwater,Long Way Down,MorningWars,2018-08-03,174999,0,spotify:user:predict0,2018-08-28T19:51:58Z,...,-11.258,0,0.0461,0.000005,0.020400,0.1150,0.477,150.042,4,8
9,65rLHt6A58MFRxlNWVDU1Z,WlYiRrlrChWktQDo,Summer,Summer,NoSo,2018-08-01,232746,22,spotify:user:predict0,2018-08-28T19:51:58Z,...,-7.517,1,0.0380,0.016800,0.007230,0.0706,0.210,123.962,4,9


In [12]:
# Convertemos o dataframe em um dicionário de músicas
dict_musicas = df_dsaminiprojeto7.to_dict(orient = "records")

In [13]:
dict_musicas[1:3]

[{'Spotify ID': '4J39ZEbwqHwtWLImUKmrn9',
  'Artist IDs': 'CRfAxYjJsDBHwvWFnjaRRRPXwFwQmoTNqNHBGU',
  'Track Name': '88 Days',
  'Album Name': 'Heat',
  'Artist Name(s)': 'SaraKingIanOlney',
  'Release Date': '2018-08-04',
  'Duration (ms)': 227961,
  'Popularity': 8,
  'Added By': 'spotify:user:predict0',
  'Added At': '2018-08-28T19:51:58Z',
  'Genres': 'bedroom pop',
  'Danceability': 0.335,
  'Energy': 0.401,
  'Key': 3,
  'Loudness': -10.749,
  'Mode': 1,
  'Speechiness': 0.0333,
  'Acousticness': 0.134,
  'Instrumentalness': 0.582,
  'Liveness': 0.134,
  'Valence': 0.233,
  'Tempo': 155.062,
  'Time Signature': 4,
  'order_id': 1},
 {'Spotify ID': '0a12d4HUjOmQSqHqLopWYx',
  'Artist IDs': 'hytHTGTflktWAhKcxQ',
  'Track Name': 'Castaway',
  'Album Name': 'Castaway',
  'Artist Name(s)': 'ARZLEE',
  'Release Date': '2018-08-10',
  'Duration (ms)': 230000,
  'Popularity': 0,
  'Added By': 'spotify:user:predict0',
  'Added At': '2018-08-28T19:51:58Z',
  'Genres': nan,
  'Danceability'

In [ ]:
# Kafka Producer
if __name__ == "__main__":

    # Cria o producer
    producer = KafkaProducer(bootstrap_servers = SERVER, 
                             value_serializer = lambda x: x.encode('utf-8'))
    
    # Variáveis de controle
    send = []    
    send = None

    # Loop pelo dicionário de músicas
    for musica in dict_musicas:
        
        # Cria a lista com dados que serão enviados para o Kafka
        sending = []
        
        # Append de cada coluna
        sending.append(musica["order_id"])
        sending.append(musica["Spotify ID"])
        sending.append(musica["Track Name"])
        sending.append(musica["Popularity"])
        sending.append(musica["Duration (ms)"])
        sending.append(musica["Artist Name(s)"])
        sending.append(musica["Artist IDs"])
        sending.append(musica["Release Date"])
        sending.append(musica["Danceability"])
        sending.append(musica["Energy"])
        sending.append(musica["Key"])
        sending.append(musica["Loudness"])
        sending.append(musica["Mode"])
        sending.append(musica["Speechiness"])
        sending.append(musica["Acousticness"])
        sending.append(musica["Instrumentalness"])
        sending.append(musica["Liveness"])
        sending.append(musica["Valence"])
        sending.append(musica["Tempo"])
        sending.append(musica["Time Signature"])
        
        # Junta tudo
        musica = ','.join(str(v) for v in sending)

        # Envia os dados para o tópico
        print("Próxima Música:" )
        print(musica)
        producer.send(TOPIC, musica)
        time.sleep(1)

    print("Concluído")

Próxima Música:
0,22a0Ji6EQKkY0tBohlN4Od,There You Are,2,231240,KirstenLudwig,qLyYYhSlsjwymwVKwW,2018-08-06,0.487,0.707,9,-5.596,0,0.0304,0.334,0.282,0.105,0.316,129.856,4
Próxima Música:
1,4J39ZEbwqHwtWLImUKmrn9,88 Days,8,227961,SaraKingIanOlney,CRfAxYjJsDBHwvWFnjaRRRPXwFwQmoTNqNHBGU,2018-08-04,0.335,0.401,3,-10.749,1,0.0333,0.134,0.582,0.134,0.233,155.062,4
Próxima Música:
2,0a12d4HUjOmQSqHqLopWYx,Castaway,0,230000,ARZLEE,hytHTGTflktWAhKcxQ,2018-08-10,0.553,0.422,1,-11.29,1,0.0314,0.11,3.25e-05,0.119,0.29,83.988,4
Próxima Música:
3,4u1DykFW1HjYAGNoDCiXfC,Arouse,30,213913,Shagabondgoodboynoah,WjyoJHRHlTbUTZTwqpAgeqmtJlARXjon,2018-08-03,0.67,0.751,1,-6.066,1,0.433,0.0728,0.0,0.368,0.533,91.961,4
Próxima Música:
4,0u7JZm9ORerlZnnxxSdMwl,Lonely,21,258738,Hayleau,AdKmjgFzpcTvmVfGwR,2018-08-10,0.67,0.709,8,-3.921,0,0.0406,0.0169,0.00063,0.0542,0.577,98.954,4
Próxima Música:
5,0wuy2BYIVLbflFDqnR9Jay,Orsay,6,413658,TheSvens,kCwrYUFSJCubbbnZrE,2018-08-03,0.61,0.444,0,-11.858,1,0.0316,0.0486,0

Próxima Música:
48,7pmOnUw96WpDqNiIi65Pkk,WFM,1,183480,HeaveeGantManDJPhilSirrTMO,bTrwZAKTLYIzozCHzxwFRfYoRwTBLHVVwzksLnxvdqgPgQjYxLHUAbGwMCACxOadCSiTVIMY,2018-07-30,0.855,0.49,6,-13.725,0,0.268,0.0724,0.184,0.0848,0.618,160.021,4
Próxima Música:
49,1p8NdPlZ98XpGyMpDeQBr3,Vague à l’Âme,2,261834,BenoitB,jlupzHcVyIzWVbDXBFunW,2018-07-27,0.683,0.674,4,-8.431,0,0.0464,0.00534,0.777,0.0872,0.449,125.498,4
Próxima Música:
50,5KeTziRHUzcL0PmtgJS0xi,unseen paths,0,207500,Sundrugs,pIpggGDYuURviXCh,2018-07-27,0.168,0.254,7,-17.357,1,0.0529,0.0103,0.919,0.189,0.0372,123.659,5
Próxima Música:
51,23dxdd9RZJrhzpz5yGZmpm,Maraka,0,226601,AyotemiJahTrue,IzXoNjKjKKRAnXTlYEYxHWhuuvaYLucjHKt,2018-07-02,0.578,0.332,10,-11.654,0,0.672,0.697,0.0,0.101,0.735,82.309,4
Próxima Música:
52,1SmauanDjICqV26Whsk4Ct,Too Fast Too Soon,0,218253,ZOLA,JofBDurgOATlfzzyniZp,2018-07-20,0.663,0.368,5,-7.974,1,0.0304,0.703,0.0,0.0778,0.433,160.231,3
Próxima Música:
53,4EltV9XnANDglO4Q8XnfJ6,Colder,0,161454,AceyGinTonic,CgvfFk

Próxima Música:
95,6jCOxaAZTmKTTMe8ZebiUM,Curb Ciggiez,4,205055,BeachBody,uHAtwbIsKXLnZGtP,2018-07-06,0.53,0.402,1,-8.814,0,0.0295,0.132,0.9,0.125,0.65,121.886,4
Próxima Música:
96,056VnZAlrZ6aFc2e4q2ZzT,Out of This Country,3,191612,Babehoven,YjrlVbArFeLmpwja,2018-07-13,0.573,0.469,4,-6.017,1,0.0255,0.00471,1.31e-05,0.108,0.347,100.224,3
Próxima Música:
97,3jH5vnBDwWzCNFDWG3ckxB,Raincheck,3,179533,ShardaiGabrielle,bZoeGRmLVsnVetcENoCF,2018-08-14,0.598,0.594,1,-9.619,0,0.05,0.346,0.00496,0.125,0.587,132.158,4
Próxima Música:
98,4tY61fdEwlHweFkW0JUPOi,Studio,0,295386,Ryahn,JyGTHjhkaEhsJQfCVAX,2018-07-13,0.812,0.399,6,-11.428,0,0.0998,0.734,0.0,0.114,0.695,129.939,4
Próxima Música:
99,3KJvQO57agV0gKONdLGAIm,Swingset,17,170000,DannyDwyer,dJMOJVjZArXLdDJNd,2018-07-19,0.706,0.63,2,-8.593,1,0.0371,0.16,0.0018,0.278,0.51,144.011,4
Próxima Música:
100,1bX1nY3TlcAhbuL4UA3tQY,FOA,0,162586,wwoman,bvaVxyLIPalRkLXBkYrX,2018-07-13,0.713,0.687,2,-8.102,1,0.0319,0.049,0.564,0.218,0.55,129.985,4
Próxima

Próxima Música:
143,5W6WQdVPFApaKv4Rg37XLh,In Waves,39,264000,neoncities,CIGExvLuDnaVTGcwTw,2018-06-19,0.432,0.729,9,-6.898,0,0.0313,0.000404,0.809,0.0979,0.305,179.987,3
Próxima Música:
144,0COfJVwsdgpIc1Ytz3WcJ0,Good Ole Urns,8,215293,Pioneer,UvRsxGVAZhrrgODHn,2018-06-22,0.527,0.534,6,-10.396,0,0.0399,0.715,0.6,0.126,0.237,87.011,4
Próxima Música:
145,5JsnMp5Pj5psasoXbKaaXa,Qualm,0,153325,HelenaHauff,JcefSOPbcWEluLiEIaN,2018-06-19,0.158,0.172,0,-14.058,1,0.0416,0.472,0.00422,0.0775,0.0711,89.116,3
Próxima Música:
146,7HKTEtjVFQrMzXzjBpTfns,Ballin,37,213000,Parisalexa,CZXfBcIhQdPEqkNvV,2018-06-08,0.705,0.371,8,-9.398,1,0.157,0.163,1.12e-06,0.0718,0.541,74.931,4
Próxima Música:
147,3YKQqS7xr8DcXggBwv4I5P,Meltaway,26,221379,Zhao,bfwydBnntONbtHagaQf,2018-06-05,0.802,0.343,5,-11.15,1,0.0512,0.269,0.0187,0.0723,0.438,115.998,4
Próxima Música:
148,4kVoojknoRgLf2X6AForTQ,Blue Valentine,30,171000,nia,KIKebHRHSmzpNkajV,2018-06-09,0.664,0.837,1,-7.429,1,0.03,0.108,0.00153,0.342,0.91,116.12,4
Pr

Próxima Música:
190,5vBrB1l5KpUnQydnfFFacZ,Hazeldene,35,246065,RoyalYellow,igwfdEsHYGtmuwy,2018-05-17,0.679,0.49,2,-8.275,1,0.03,0.106,0.357,0.111,0.615,81.494,4
Próxima Música:
191,5v4zDgMv6ca9CWwfCyUz5O,Bailee,40,196153,TheLicks,ENHpFyXQwTNkRarHKy,2018-05-12,0.677,0.808,9,-7.729,1,0.0479,0.0288,0.00956,0.112,0.551,145.025,4
Próxima Música:
192,2Gdij9TUSeRPDMF1hXuTML,East End,12,191949,CloudyRetro,RiJYgMefrEBRrfZymuB,2018-05-11,0.852,0.273,9,-13.272,0,0.0678,0.594,0.000246,0.129,0.398,133.948,4
Próxima Música:
193,7oJmIBxEAYjfkS7lhZ7M1S,Sometimes,62,262736,GothBabe,oHOzrujyATtVsqGhh,2018-05-07,0.445,0.563,7,-8.904,1,0.0315,0.105,0.000271,0.179,0.204,94.978,4
Próxima Música:
194,5bHyCIt6Ho54FNmJSDYtaM,Complete Sense,3,135012,LateBloomersInternet,ZneONkaYnpMlNwBsUYRWLxNBGMyWfCjnubLF,2018-05-09,0.516,0.617,1,-9.004,1,0.0267,0.153,0.00233,0.105,0.922,104.588,4
Próxima Música:
195,3sy0Fnsw3hp8EfU1yWIKYR,Spotlight,34,154800,NearxFarAwa,jHNFPilHsebvSIGwdSmGLpPwNXREZFCZKP,2018-05-16,0.757,0.5

Próxima Música:
238,31b314XCYco5cgN8qEUcHI,Domino,0,174289,CaineCasket,bfSqqLwtuezcLdxvHv,2018-05-18,0.665,0.409,1,-12.489,0,0.343,0.145,5.19e-05,0.0909,0.442,156.944,4
Próxima Música:
239,26XZbxgQjGyhMoVgWPBH0P,Misfit,0,181685,Naji,QMdBtewZLphCnXKK,2018-05-25,0.531,0.512,11,-10.788,0,0.439,0.475,0.0,0.362,0.35,75.968,4
Próxima Música:
240,5DCjd4G5JeaAwkbqiGvgI5,Vegas Girl,0,161935,JamieBlakeMistaDC,QKLlDrLWdImMmAhVdUeOcsrgakaREzYveNQsc,2018-05-21,0.623,0.622,5,-9.28,0,0.137,0.152,0.0,0.142,0.178,112.165,4
Próxima Música:
241,4hcGyAi0G02kxjoNXkjMwS,Flute,0,215409,WantiggaSamTibaCanblaster,jAydcjNjyqbeVYLUImFJTutxWjPAnFigeyCbIUKzbOAkiMVUaeWk,2018-05-25,0.801,0.594,10,-7.345,0,0.0394,0.139,0.723,0.128,0.324,137.009,4
Próxima Música:
242,5qtXDrg2VApdmYPJuDkfiw,look,13,274000,iivvyy,UnmIEYPLlOTUDWAHaMH,2018-05-16,0.547,0.595,10,-6.382,0,0.0587,0.075,0.00891,0.0956,0.215,119.932,4
Próxima Música:
243,4Bs4eukYAlBW5yZ7sF5ZP6,HXD,11,193256,LittleSnake,HrvmFozKYzLjPBZfX,2018-05-25,0.755,0.956,1

Próxima Música:
286,4YY8nv8tzU7AKLyRxTP23I,You,4,131152,GylesBartle,KQZeCYZnzaCuCSbQIP,2018-05-11,0.607,0.331,0,-14.866,1,0.177,0.268,0.327,0.363,0.663,72.083,4
Próxima Música:
287,7fAJw7CNYOOMbYr3Sr3XwJ,Amor Fati,0,214884,ChampagneSuperchillin,jcUpTfktZTfXtKgtPx,2018-05-10,0.362,0.841,2,-6.682,0,0.0401,0.123,0.118,0.117,0.689,118.334,4
Próxima Música:
288,4SzpxKCWRcKNNce1JU4Gag,Georgie Wants a Garden,9,183533,Blackaby,WopVJNPzVzDbVbPJQwE,2018-05-11,0.711,0.573,11,-8.577,1,0.0282,0.819,0.0556,0.118,0.823,113.054,4
Próxima Música:
289,40Ub2OAfCIxJa7OaOJlGxw,So Simple,6,216173,LucyandtheRats,ASxyxLrmWqaSjxxiEu,2018-05-07,0.343,0.881,4,-4.169,1,0.0306,0.00628,0.000164,0.0887,0.444,151.737,4
Próxima Música:
290,2RnTn9CS7jb6TcDjLBwn9Y,reasons,19,185142,wishedbone,PpncOYFkaKQZcONdmc,2018-05-11,0.665,0.366,3,-10.908,1,0.037,0.848,0.802,0.107,0.283,111.772,4
Próxima Música:
291,6WhXE4p7k0Hn0TsAGR0zgw,Baptist,1,258500,Lucas,JudMLOeZkdPAIUBW,2018-05-11,0.578,0.43,6,-7.25,0,0.0724,0.397,0.05,0.08

Próxima Música:
334,1X3GHmCJQ9v53Ll6CLuUZJ,Sexy,0,173322,Jilly,wlpLtccOLHzNvPjVjon,2018-04-23,0.719,0.737,0,-6.631,1,0.0873,0.24,0.00128,0.161,0.789,159.068,4
Próxima Música:
335,35YU8j2PEDAFkg3sZUarjR,Own2,14,217493,AprilVISTA,XNCSlVEpHHbaHvEabAj,2018-02-23,0.863,0.447,4,-5.367,0,0.0512,0.312,4.07e-05,0.247,0.683,115.02,4
Próxima Música:
336,6q44hsVA6PwiK9dB5Rv1qE,Circle of Life,0,169014,LunarRAE,RRiuEQfALMcOZMrL,2018-04-20,0.56,0.668,8,-6.808,1,0.313,0.0168,0.000178,0.166,0.553,70.862,4
Próxima Música:
337,2vAoAIKiU0lcEmHHJe2Iih,New Crush,10,179919,MoonRacer,JdveWSwhPYpHKWcKw,2018-04-20,0.713,0.43,0,-12.76,1,0.0388,0.738,0.939,0.0922,0.775,131.789,4
Próxima Música:
338,116zQaQEsas1cKdknWB1m2,Versions,19,195390,StelloScottJames,DwxIEnUHlgIuQSuFFTAdTBZIUQCRwIwdADy,2018-04-23,0.395,0.625,0,-9.181,0,0.0501,0.777,0.00152,0.311,0.424,107.012,4
Próxima Música:
339,2HkwJWMMo1NBXqV7QZkNky,I Can't Remember,3,234600,MattDorrien,kpxoeEMLlSxqfbEB,2018-04-20,0.572,0.276,7,-10.478,1,0.0283,0.864,0.

Próxima Música:
382,7qlMj4eVH2KWUFHJ0yFLS9,ODO,0,202005,ProjectParadisMrCarmackPromnite,knGxcpecLosNwAOOHaWpWPKDcZgOOUnelFnnhrHJLtBXqhhliwceEI,2018-04-20,0.578,0.381,6,-11.385,0,0.108,0.0877,0.639,0.18,0.106,151.002,4
Próxima Música:
383,2aRWNus6CXlJGyzL8vOXW9,East Bay Grime,1,150720,Onhell,SJhirDGiBwrtfvuat,2018-04-20,0.765,0.895,1,-4.7,1,0.38,0.132,0.286,0.304,0.429,142.008,3
Próxima Música:
384,6Y2NnUmkMjXt6F5nkgZ9Xj,ru a samurai?,1,240076,LISACHRIS,EwjchAbfUOxYbxMr,2018-04-20,0.722,0.6,10,-9.4,0,0.0548,0.0205,0.837,0.124,0.398,155.991,4
Próxima Música:
385,7LamR9pjLH3vYQSaUyuIa0,Redwood,4,253125,Cass,UBDJpGBiyvUHRhgcf,2018-04-16,0.165,0.229,0,-15.957,0,0.0332,0.985,0.582,0.357,0.0387,87.532,1
Próxima Música:
386,2Ld8cSHwZCqJAaN77vMVNd,Delirious,25,205892,HaleyDaniels,HRphoUIrSZxCuhhUC,2018-04-06,0.68,0.394,10,-10.931,1,0.058,0.784,0.00724,0.119,0.434,120.505,4
Próxima Música:
387,4e2LfPZ9lVdWsQVWYdYzo4,Something, in General,38,240937,RittMomney,MPGCBQoalYMOPkBka,2018-04-06,0.487,0.

Próxima Música:
430,5bUxsWmgIzlemyEVvLBqrO,WYA,0,259987,ChrisLaBellaRaShaun,xXFNOuKKjVkPBpYXeTkWCeZRsLpyZFoyoSnpEt,2018-03-30,0.718,0.506,0,-9.289,1,0.142,0.174,1.32e-05,0.111,0.175,119.933,4
Próxima Música:
431,3JzsyosUPeRY5yOhsNTn1o,My First Ghost,6,318000,SenMorimoto,zlWvRPwLwGSivltrhOl,2018-03-25,0.591,0.481,0,-7.464,1,0.0364,0.772,0.0667,0.0934,0.381,120.028,4
Próxima Música:
432,5lnOQeuqK88uBy2HsRRivq,MEANT 2 BE,0,203414,MichaelParadise,fHMYXpFpnKsPvyss,2018-03-30,0.514,0.69,9,-6.24,0,0.0344,0.639,0.798,0.192,0.586,82.009,4
Próxima Música:
433,7ivYikkPEulfUGreJdTiXv,Washed Away,2,184122,Ravennas,tUuEkEsKpnTysFONwT,2018-03-28,0.435,0.9,11,-4.75,0,0.0451,0.0283,0.533,0.537,0.604,131.04,4
Próxima Música:
434,5OANa4wh3pZsu2nW7paD5v,Never Will,0,232315,Headclouds,NbjjKpZKESQpXGdTz,2018-03-30,0.513,0.634,7,-6.913,1,0.0354,0.0185,0.00431,0.3,0.346,121.928,4
Próxima Música:
435,05UX5TG1K5a9AKGGpYOpNT,SOFT SCOOP - Iced Coma Mix,0,191884,Scent,jBkGCRcgimHCmSrlicE,2018-03-27,0.44,0.622,0,-1

# Fim